In [ ]:
import numpy as np
import tensorflow as tf
from nbody import getAcc_naive, getAcc_numpy

In [ ]:
# Simulation parameters
N = 1000  # Number of particles
t = 0  # current time of the simulation
tEnd = 10.0  # time at which simulation ends
dt = 0.01  # timestep
softening = 0.1  # softening length
G = 1.0  # Newton's Gravitational Constant
plotRealTime = True  # switch on for plotting as the simulation goes along

In [ ]:
# Generate Initial Conditions
np.random.seed(17)  # set the random number generator seed

mass = 20.0 * np.ones((N, 1)) / N  # total mass of particles is 20
mass_sq = np.squeeze(mass)
pos = np.random.randn(N, 3)  # randomly selected positions and velocities
vel = np.random.randn(N, 3)

# Convert to Center-of-Mass frame
vel -= np.mean(mass * vel, 0) / np.mean(mass)

In [ ]:
# %%timeit
N = pos.shape[0]
a = np.zeros((N, 3))

for i in range(N):
	for j in range(N):
		dx = pos[j, 0] - pos[i, 0]
		dy = pos[j, 1] - pos[i, 1]
		dz = pos[j, 2] - pos[i, 2]
		inv_r3 = (dx ** 2 + dy ** 2 + dz ** 2 + softening ** 2) ** (-1.5)
		a[i, 0] += G * (dx * inv_r3) * mass[j]
		a[i, 1] += G * (dy * inv_r3) * mass[j]
		a[i, 2] += G * (dz * inv_r3) * mass[j]
a2 = a.copy()

### Vectorization

In [ ]:
A = np.random.rand(1024, 1024)
B = np.random.rand(1024, 1024)

### Apply vectorization

In [ ]:
(np.abs(a - a2)).sum()

In [ ]:
%%timeit
a = get_acc_numpy_one_loop(pos, mass, G, softening)

### Full vectorization

In [ ]:
def get_acc_numpy(pos, mass, G, softening):
    """
    Calculate the acceleration on each particle due to Newton's Law 
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    # positions r = [x,y,z] for all particles
    x = pos[:, 0:1]
    y = pos[:, 1:2]
    z = pos[:, 2:3]

    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z

    # matrix that stores 1/r^3 for all particle pairwise particle separations
    inv_r3 = (dx ** 2 + dy ** 2 + dz ** 2 + softening ** 2)
    inv_r3[inv_r3 > 0] = inv_r3[inv_r3 > 0] ** (-1.5)

    ax = G * (dx * inv_r3) @ mass
    ay = G * (dy * inv_r3) @ mass
    az = G * (dz * inv_r3) @ mass

    # pack together the acceleration components
    a = np.hstack((ax, ay, az))

    return a

In [ ]:
%%timeit
a = get_acc_numpy(pos, mass, G, softening)

### Quick performance test - numpy vs TF

In [ ]:
np_test_arr = np.random.randn(1024, 1024)
tf_test_arr = tf.convert_to_tensor(np_test_arr)

In [ ]:
%%timeit
a = tf.reduce_mean(tf_test_arr)

In [ ]:
%%timeit
b = np.mean(np_test_arr)

In [ ]:
%%timeit
a = tf.reduce_max(tf_test_arr)

In [ ]:
%%timeit
b = np.max(np_test_arr)

### Tensorflow implementation

In [ ]:
pos_tf = tf.convert_to_tensor(pos)
mass_tf = tf.convert_to_tensor(mass)
G_tf = tf.convert_to_tensor(G, dtype=tf.float64)
softening_tf = tf.convert_to_tensor(softening, dtype=tf.float64)

In [ ]:
%%timeit

# positions r = [x,y,z] for all particles
x = pos_tf[:, 0:1]
y = pos_tf[:, 1:2]
z = pos_tf[:, 2:3]

# matrix that stores all pairwise particle separations: r_j - r_i
dx = tf.transpose(x) - x
dy = tf.transpose(y) - y
dz = tf.transpose(z) - z

inv_r3 = (dx ** 2 + dy ** 2 + dz ** 2 + softening_tf ** 2) ** (-1.5)

ax = G_tf * (dx * inv_r3) @ mass_tf
ay = G_tf * (dy * inv_r3) @ mass_tf
az = G_tf * (dz * inv_r3) @ mass_tf

a = tf.concat([ax, ay, az], axis=1)

In [ ]:
def get_acc_tf(pos_tf, mass_tf, G_tf, softening_tf):
    # positions r = [x,y,z] for all particles
    x = pos_tf[:, 0:1]
    y = pos_tf[:, 1:2]
    z = pos_tf[:, 2:3]

    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = tf.transpose(x) - x
    dy = tf.transpose(y) - y
    dz = tf.transpose(z) - z

    inv_r3 = (dx ** 2 + dy ** 2 + dz ** 2 + softening_tf ** 2) ** (-1.5)

    ax = G_tf * (dx * inv_r3) @ mass_tf
    ay = G_tf * (dy * inv_r3) @ mass_tf
    az = G_tf * (dz * inv_r3) @ mass_tf

    return tf.concat([ax, ay, az], axis=1)

get_acc_tf_graph = tf.function(get_acc_tf)

In [ ]:
%%timeit
b = get_acc_tf_graph(pos_tf, mass_tf, G_tf, softening_tf)

### Data types optimisation

In [ ]:
pos32 = pos.astype(np.float32)
mass32 = mass.astype(np.float32)

In [ ]:
%%timeit
### NUMPY
a = get_acc_numpy(pos32, mass32, G, softening)

In [ ]:
pos_tf_32 = tf.convert_to_tensor(pos, dtype=tf.float32)
mass_tf_32 = tf.convert_to_tensor(mass, dtype=tf.float32)
G_tf_32 = tf.convert_to_tensor(G, dtype=tf.float32)
softening_tf_32 = tf.convert_to_tensor(softening, dtype=tf.float32)

In [ ]:
%%timeit
b = get_acc_tf_graph(pos_tf_32, mass_tf_32, G_tf_32, softening_tf_32)